In [1]:
%pip install librosa matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt
import torch ## 저같은 경우엔 pytorch를 사용하여 학습하고 있어서 torch tensor로 저장하고자합니다
from tqdm import tqdm
import IPython.display

In [4]:
import numpy as np
from PIL import Image
import cv2
import os

def augment_image(image_path, output_dir):
    original_image = Image.open(image_path)
    base_name = os.path.splitext(os.path.basename(image_path))[0]

    for angle in [0, 45, 90, 135, 180, 225, 270, 315]:
        rotated_image = original_image.rotate(angle)

        # Convert PIL Image to OpenCV format
        cv_image = cv2.cvtColor(np.array(rotated_image), cv2.COLOR_RGB2BGR)

        # Flip
        flipped_image = cv2.flip(cv_image, 1)  # 1 for horizontal flip
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_flipped.jpg"), flipped_image)

        # Median Blur
        blurred_image = cv2.medianBlur(cv_image, 5)  # 5 is the kernel size
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_blurred.jpg"), blurred_image)

        # Noise
        noise = np.random.normal(0, 25, cv_image.shape).astype(np.uint8)
        noisy_image = cv2.add(cv_image, noise)
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_noisy.jpg"), noisy_image)

        # Save the original rotated image
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}.jpg"), cv_image)

def augment_all_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir, filename)
            augment_image(input_path, output_dir)

# 사용 예시
augment_all_images('/root/data/train', '/root/data/augmented_train')

In [19]:
import csv
import os

def update_csv(original_csv_path, augmented_image_dir, output_csv_path):
    # 원본 CSV 파일 읽기
    with open(original_csv_path, 'r') as f:
        reader = csv.DictReader(f)
        original_data = list(reader)

    # 새로운 데이터를 저장할 리스트
    new_data = []

    # 증강된 이미지에 대한 데이터 추가
    for row in original_data:
        original_filename = row['ID']
        target = row['target']

        # 파일 이름에서 확장자 제거
        base_name = os.path.splitext(original_filename)[0]

        for angle in [0, 45, 90, 135, 180, 225, 270, 315]:
            # 회전된 이미지
            rotated_filename = f"{base_name}_rotated_{angle}.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, rotated_filename)):
                new_data.append({
                    'ID': rotated_filename,
                    'target': target
                })

            # Flipped 이미지
            flipped_filename = f"{base_name}_rotated_{angle}_flipped.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, flipped_filename)):
                new_data.append({
                    'ID': flipped_filename,
                    'target': target
                })

            # Blurred 이미지
            blurred_filename = f"{base_name}_rotated_{angle}_blurred.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, blurred_filename)):
                new_data.append({
                    'ID': blurred_filename,
                    'target': target
                })

            # Noisy 이미지
            noisy_filename = f"{base_name}_rotated_{angle}_noisy.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, noisy_filename)):
                new_data.append({
                    'ID': noisy_filename,
                    'target': target
                })

    # 새로운 CSV 파일 작성
    with open(output_csv_path, 'w', newline='') as f:
        fieldnames = ['ID', 'target']
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        writer.writeheader()
        for row in new_data:
            writer.writerow(row)

    print(f"Updated CSV file with only augmented data has been saved to {output_csv_path}")

# 사용 예시
update_csv('/root/data/train.csv', '/root/data/augmented_train', '/root/data/train_augmented.csv')

Updated CSV file with only augmented data has been saved to /root/data/train_augmented.csv


In [15]:
# 두 폴더(/home/data/train, /home/data/augmented_train)의 이미지 파일 합쳐서 하나의 새로운 폴더(/home/data/rotate_train)로 이동
import os
import shutil

def move_images(input_dir1, input_dir2, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir1):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir1, filename)
            output_path = os.path.join(output_dir, filename)
            shutil.copy(input_path, output_path)

    for filename in os.listdir(input_dir2):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir2, filename)
            output_path = os.path.join(output_dir, filename)
            shutil.copy(input_path, output_path)

move_images('/root/data/train', '/root/data/augmented_train', '/root/data/rotate_train')

In [17]:
train_path = "/root/data/rotate_tarin"
cnt = 0
for root, dirs, files in os.walk(train_path):
    cnt += len(files)

print(f"Total number of files in {train_path}: {cnt}")

Total number of files in /root/data/rotate_tarin: 0


In [21]:
import pandas as pd

data = "/root/data/train_augmented.csv"
data = pd.read_csv(data)
data.head(20)

,ID,target
0,06c04208033a0ebf_rotated_0.jpg,0
1,06c04208033a0ebf_rotated_0_flipped.jpg,0
2,06c04208033a0ebf_rotated_0_blurred.jpg,0
3,06c04208033a0ebf_rotated_0_noisy.jpg,0
4,06c04208033a0ebf_rotated_45.jpg,0
5,06c04208033a0ebf_rotated_45_flipped.jpg,0
6,06c04208033a0ebf_rotated_45_blurred.jpg,0
7,06c04208033a0ebf_rotated_45_noisy.jpg,0
8,06c04208033a0ebf_rotated_90.jpg,0
9,06c04208033a0ebf_rotated_90_flipped.jpg,0


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50240 entries, 0 to 50239
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50240 non-null  object
 1   target  50240 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 785.1+ KB
